In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-price/1553768847-housing.csv


In [53]:
df = pd.read_csv("/kaggle/input/house-price/1553768847-housing.csv")
is_missing_attr = df.isna()
n_missing_attr = is_missing_attr.sum(axis=1)
df[n_missing_attr > 0]





,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
290,-122.16,37.77,47,1256,NaN,570,218,4.3750,NEAR BAY,161900
341,-122.17,37.75,38,992,NaN,732,259,1.6196,NEAR BAY,85100
538,-122.28,37.78,29,5154,NaN,3741,1273,2.5762,NEAR BAY,173400
563,-122.24,37.75,45,891,NaN,384,146,4.9489,NEAR BAY,247100
696,-122.10,37.69,41,746,NaN,387,161,3.9063,NEAR BAY,178400
...,...,...,...,...,...,...,...,...,...,...
20267,-119.19,34.20,18,3620,NaN,3171,779,3.3409,NEAR OCEAN,220500
20268,-119.18,34.19,19,2393,NaN,1938,762,1.6953,NEAR OCEAN,167400
20372,-118.88,34.17,15,4260,NaN,1701,669,5.1033,<1H OCEAN,410700
20460,-118.75,34.29,17,5512,NaN,2734,814,6.6073,<1H OCEAN,258100


In [ ]:
df.dropna(inplace=True)
df


# shuffling DS

In [ ]:
shuffled = df.sample(frac=1, random_state=1).reset_index(drop=True)
shuffled

# splitting

In [ ]:

from sklearn.model_selection import train_test_split

trainval, test = train_test_split(shuffled, test_size=0.16, shuffle=False)
train, val = train_test_split(trainval, test_size=0.2, shuffle=False)


df_target_train = train['median_house_value']
train.drop("median_house_value",axis=1,inplace=True)

df_target_test = test['median_house_value']
test.drop("median_house_value",axis=1,inplace=True)

df_target_val = val['median_house_value']
val.drop("median_house_value",axis=1,inplace=True)

In [ ]:
len(test),len(train),len(val)
print(val["ocean_proximity"].unique())
print(train["ocean_proximity"].unique()) 
print(test["ocean_proximity"].unique())
# no dataset de trainamento tem um ISLAND que nao tem no test e validation. 

island_regs = train[ train["ocean_proximity"] == "ISLAND"]
train.drop(island_regs.index,inplace=True)



In [ ]:
print(val["ocean_proximity"].unique())
print(train["ocean_proximity"].unique()) 
print(test["ocean_proximity"].unique())

# Normaling dataset
splitting data in categorical, ordinal and continuous

In [ ]:
display(train)
contin_attr = ["housing_median_age","total_rooms","total_bedrooms","population","households","median_income"]
cate_attr = ["ocean_proximity"]

# Normalizing continious values

In [ ]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [ ]:
dataset = {"train":None, "test":None, "val":None}

scaler = StandardScaler()
encoder = OneHotEncoder(sparse=False)

# Fit the scaler on the training data
scaler.fit(train[contin_attr])

encoder = OneHotEncoder(sparse=False)


def normalizing(df): 
    return scaler.transform(df)

def encoding(df):
    return encoder.fit_transform(df)

In [ ]:
train_cont = pd.DataFrame(normalizing(train[contin_attr])) 
train_cat = pd.DataFrame(encoding(train[cate_attr]))
dataset["train"] = pd.concat([train_cont,train_cat],axis=1)

test_cont = pd.DataFrame(normalizing(test[contin_attr])) 
test_cat = pd.DataFrame(encoding(test[cate_attr]))
dataset["test"] = pd.concat([test_cont,test_cat],axis=1)

val_cont = pd.DataFrame(normalizing(val[contin_attr])) 
val_cat = pd.DataFrame(encoding(val[cate_attr]))
dataset["val"] = pd.concat([val_cont,val_cat],axis=1)

In [ ]:
len(dataset["train"]),len(dataset["test"]),len(dataset["val"])
dataset["train"]

# dataset

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np


In [ ]:
# Convert numpy arrays to PyTorch tensors
features_tensor = torch.tensor(dataset["train"].to_numpy(),dtype=torch.float32)
target_tensor = torch.tensor(df_target_train.to_numpy(),dtype=torch.float32)

# VALIDATION
val_features_tensor = torch.tensor(dataset["val"].to_numpy(),dtype=torch.float32)
val_target_tensor = torch.tensor(df_target_val.to_numpy(),dtype=torch.float32)


In [ ]:
tensor_dataset = TensorDataset(features_tensor, target_tensor)
dataloader = DataLoader(tensor_dataset, batch_size=32, shuffle=True)


#VALIDATION
val_tensor_dataset = TensorDataset(val_features_tensor, val_target_tensor)
val_dataloader = DataLoader(val_tensor_dataset, batch_size=32, shuffle=True)


In [ ]:
# Step 4: Iterate through the DataLoader
for batch_features, batch_targets in dataloader:
    print("Batch features:", batch_features)
    print("Batch targets:", batch_targets)
    break  # Just to show the first batch


# Linear Regression

In [ ]:
import torch.nn as nn

torch.manual_seed(13)
linear = nn.Linear(in_features=11, out_features=1, bias=True)

model = nn.Sequential()
model.add_module('layer1', linear)
model.state_dict(),list(model.parameters())


In [ ]:
bad_predictions = model(batch_features)
bad_predictions

In [ ]:
bad_predictions - batch_targets


## Loss - MSE

In [ ]:
loss_fn = nn.MSELoss()
loss = loss_fn(bad_predictions, batch_targets)
loss

In [ ]:
# aplicando o gradient descent automatico
loss.backward()
model.layer1.weight.grad, model.layer1.bias.grad
# -470863.1250 é uma tx de erro muito menor que 6.6361e+10

# Optimizer 

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(params=model.parameters(), lr=1.1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
n_epochs = 100
losses = torch.empty(n_epochs) 
model.to(device) 

In [ ]:
for epoch in range(n_epochs):
    batch_losses = torch.empty(len(dataloader)) 
    #print(epoch)
    for i, (batch_features, batch_targets) in enumerate(dataloader):
        model.train()
        batch_features = batch_features.to(device)
        batch_targets = batch_targets.to(device) 

        # Step 1 - forward pass
        predictions = model(batch_features) 

        # Step 2 - computing the loss
        loss = loss_fn(predictions, batch_targets) 

        # Step 3 - computing the gradients
        loss.backward()
        batch_losses[i] = loss.item() 

        # Step 4 - updating parameters and zeroing gradients
        optimizer.step()
        optimizer.zero_grad() 

    losses[epoch] = batch_losses.mean()

## Mostrando a curva de aprendizado. Entre 900 e 1000 comecou a entrar em flat. 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')

In [ ]:
for epoch in range(n_epochs):
    batch_losses = torch.empty(len(dataloader)) 
    #print(epoch)
    for i, (batch_features, batch_targets) in enumerate(dataloader):
        model.train()
        batch_features = batch_features.to(device)
        batch_targets = batch_targets.to(device) 

        # Step 1 - forward pass
        predictions = model(batch_features) 

        # Step 2 - computing the loss
        loss = loss_fn(predictions, batch_targets) 

        # Step 3 - computing the gradients
        loss.backward()
        batch_losses[i] = loss.item() 

        # Step 4 - updating parameters and zeroing gradients
        optimizer.step()
        optimizer.zero_grad() 

    losses[epoch] = batch_losses.mean()
    
    ## VALIDATION
    batch_losses = torch.empty(len(val_dataloader)) 
    
    with torch.inference_mode():
        for i, (val_features, val_targets) in enumerate(val_dataloader):
            # Sprinkle
            model.eval() # Chamando o Eval() para ativar o evaluation

            val_features= val_features.to(device)
            val_targets = val_targets.to(device) 

            # Step 1 - forward pass
            predictions = model(val_features) 

            # Step 2 - computing the loss
            loss = loss_fn(predictions, val_targets)
            batch_losses[i] = loss.item() 

    val_losses[epoch] = batch_losses.mean()

In [ ]:
plt.plot(losses, label='Training')
plt.plot(val_losses, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.yscale('log')
plt.legend()